In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException

In [2]:
start = time.time()
# 데이터 수집 함수
def get_data(driver, genre, link):
    try:
        driver.get(link)
        wait = WebDriverWait(driver, 10)

        startdate_xpath = '//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[2]/div[1]/ul/li[1]/div/div/a/div/div[2]/div[2]/span'
        wait.until(EC.presence_of_element_located((By.XPATH, startdate_xpath)))
        start_date = driver.find_element(By.XPATH, startdate_xpath).text

        genre_xpath = '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/div[1]/div[1]/div'
        wait.until(EC.presence_of_element_located((By.XPATH, genre_xpath)))

        title_xpath = '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/span[1]'
        wait.until(EC.presence_of_element_located((By.XPATH, title_xpath)))
        title = driver.find_element(By.XPATH, title_xpath).text

        author_xpath = '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/span[2]'
        wait.until(EC.presence_of_element_located((By.XPATH, author_xpath)))
        author = driver.find_element(By.XPATH, author_xpath).text

        point_xpath = '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/div[1]/div[3]/span'
        wait.until(EC.presence_of_element_located((By.XPATH, point_xpath)))
        point = driver.find_element(By.XPATH, point_xpath).text

        view_xpath = '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/div[1]/div[2]/span'
        wait.until(EC.presence_of_element_located((By.XPATH, view_xpath)))
        view = driver.find_element(By.XPATH, view_xpath).text

        comment_xpath = '//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[3]/div/div[1]/div[1]/span'
        wait.until(EC.presence_of_element_located((By.XPATH, comment_xpath)))
        comment = driver.find_element(By.XPATH, comment_xpath).text

        count_css = '#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1 > div.mb-28pxr.ml-4px.flex.w-632pxr.flex-col.overflow-hidden.rounded-12pxr > div.flex-1.flex.flex-col > div.rounded-b-12pxr.bg-bg-a-20 > div.flex.h-44pxr.w-full.flex-row.items-center.justify-between.bg-bg-a-20.px-18pxr > div.flex.h-full.flex-1.items-center.space-x-8pxr > span'
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, count_css)))
        count = driver.find_element(By.CSS_SELECTOR, count_css).text

        sort_xpath = '//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div/span'
        driver.find_element(By.XPATH, sort_xpath).click()
        sort_desc_xpath = '/html/body/div[2]/div[3]/div[1]/div[3]/div[2]/div'
        driver.find_element(By.XPATH, sort_desc_xpath).click()
        time.sleep(0.5)

        lastdate_xpath = '//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[2]/div[1]/ul/li[1]/div/div/a/div/div[2]/div[2]/span'
        last_date = driver.find_element(By.XPATH, lastdate_xpath).text

        return {
            '제목': title,
            '장르': genre,
            '작가': author,
            '조회수': view,
            '댓글': comment,
            '연재수': count,
            '연재 시작일': start_date,
            '최신 연재일': last_date,
            '평점': point
        }
    except (NoSuchElementException, TimeoutException):
        return {
            '제목': '19금',
            '장르': '19금',
            '작가': '19금',
            '조회수': '19금',
            '댓글': '19금',
            '연재수': '19금',
            '연재 시작일': '19금',
            '최신 연재일': '19금',
            '평점': '19금'
        }

# 크롬 열기
driver = webdriver.Chrome()

# 장르별 페이지 (판타지, 현대판타지, 무협, 드라마, 로맨스판타지, 로맨스)
target_sites = [
    'https://page.kakao.com/menu/10011/screen/91', 
    'https://page.kakao.com/menu/10011/screen/64', 
    'https://page.kakao.com/menu/10011/screen/70', 
    'https://page.kakao.com/menu/10011/screen/100', 
    'https://page.kakao.com/menu/10011/screen/92', 
    'https://page.kakao.com/menu/10011/screen/68'
]

genre_map = {
    'https://page.kakao.com/menu/10011/screen/91': '판타지',
    'https://page.kakao.com/menu/10011/screen/64': '현대판타지',
    'https://page.kakao.com/menu/10011/screen/70': '무협',
    'https://page.kakao.com/menu/10011/screen/100': '드라마',
    'https://page.kakao.com/menu/10011/screen/92': '로맨스판타지',
    'https://page.kakao.com/menu/10011/screen/68': '로맨스'
}

data = []

for target_site in target_sites:
    driver.get(target_site)
    wait = WebDriverWait(driver, 10)
    
    # 하이퍼링크 저장
    links = []
    for i in range(8): # 가장 아래까지 스크롤
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(1)
    
    for i in range(1, 301):
        time.sleep(0.3)
        try:
            element = f'#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div > div.flex.grow.flex-col > div:nth-child(5) > div > div.flex.w-full.grow.flex-col > div > div > div > div:nth-child({i}) > div > a'
            link = driver.find_element(By.CSS_SELECTOR, element).get_attribute('href')
            links.append(link)
        except:
            break

    genre = genre_map[target_site]
    
    for i, link in enumerate(links):
        result = get_data(driver, genre, link)
        data.append(result)
        print(f"{genre}의 {i+1}번, 제목: {result['제목']}, 작가: {result['작가']}, 평점: {result['평점']}, 연재수: {result['연재수']}, 조회수: {result['조회수']}, 댓글수: {result['댓글']}, 연재시작일: {result['연재 시작일']}, 최근 업데이트: {result['최신 연재일']}")

driver.quit()

# 데이터프레임 생성
df = pd.DataFrame(data)
df.index = df.index + 1




판타지의 1번, 제목: 환생자의 스트리밍, 작가: 흑아인, 평점: 9.5, 연재수: 전체 122, 조회수: 92.2만, 댓글수: 전체 742, 연재시작일: 24.06.29, 최근 업데이트: 24.06.30
판타지의 2번, 제목: 마법을 품다, 작가: 양강, 평점: 9.4, 연재수: 전체 139, 조회수: 418.6만, 댓글수: 전체 1,575, 연재시작일: 24.06.05, 최근 업데이트: 24.06.30
판타지의 3번, 제목: 마법학교 마법사로 살아가는 법, 작가: 글쓰는기계, 평점: 9.9, 연재수: 전체 1,079, 조회수: 1.8억, 댓글수: 전체 16.7만, 연재시작일: 21.08.17, 최근 업데이트: 24.06.30
판타지의 4번, 제목: 천재 궁수의 스트리밍, 작가: 멍멍킴, 평점: 9.9, 연재수: 전체 981, 조회수: 2억, 댓글수: 전체 15.2만, 연재시작일: 21.02.09, 최근 업데이트: 24.06.28
판타지의 5번, 제목: 변경백 서자는 황제였다, 작가: 기준석, 평점: 10.0, 연재수: 전체 814, 조회수: 9,461.6만, 댓글수: 전체 10.5만, 연재시작일: 21.11.17, 최근 업데이트: 24.06.28
판타지의 6번, 제목: 달빛조각사: 아빠는 전설이다, 작가: 남희성, 평점: 9.5, 연재수: 전체 126, 조회수: 994.6만, 댓글수: 전체 6,893, 연재시작일: 23.12.07, 최근 업데이트: 24.06.27
판타지의 7번, 제목: 템빨, 작가: 박새날, 평점: 9.6, 연재수: 전체 2,060, 조회수: 11.8억, 댓글수: 전체 63.8만, 연재시작일: 14.12.08, 최근 업데이트: 24.04.26
판타지의 8번, 제목: 환생한 암살자는 검술 천재, 작가: 글개미, 평점: 9.8, 연재수: 전체 950, 조회수: 2.8억, 댓글수: 전체 13.4만, 연재시작일: 21.09.29, 최근 업데이트: 24.06.28
판타지의 9번, 제목: 마법학교 앞점멸 천재가 되었다, 작가: 은밀히, 평점: 9.7

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
end = time.time() # 6843.142942905426초 대략 1시간 55분 정도
print(end-start)

# 결과 출력
print(df)

6843.142942905426
                    제목   장르     작가       조회수        댓글       연재수    연재 시작일  \
1               마법을 품다  판타지     양강    311.2만  전체 1,100    전체 131  24.06.05   
2          천재 궁수의 스트리밍  판타지    멍멍킴        2억  전체 15.2만    전체 979  21.02.09   
3         변경백 서자는 황제였다  판타지    기준석  9,391.6만  전체 10.5만    전체 812  21.11.17   
4     마법학교 마법사로 살아가는 법  판타지  글쓰는기계      1.8억  전체 16.7만  전체 1,075  21.08.17   
5      달빛조각사: 아빠는 전설이다  판타지    남희성    956.5만  전체 6,723    전체 125  23.12.07   
...                ...  ...    ...       ...       ...       ...       ...   
1782               19금  로맨스    19금       19금       19금       19금       19금   
1783               19금  로맨스    19금       19금       19금       19금       19금   
1784               19금  로맨스    19금       19금       19금       19금       19금   
1785               19금  로맨스    19금       19금       19금       19금       19금   
1786               19금  로맨스    19금       19금       19금       19금       19금   

        최신 연재일    평점  
1     24.06.26   9.6  

In [ ]:
df.to_csv('./data/Kakao_장르별_Top')